In [1]:
class Person():
    def __init__(self, etternavn, fornavn, adresse, postnummer, poststed):
        self.etternavn = etternavn
        self.fornavn = fornavn
        self.adresse = adresse
        self.postnummer = postnummer
        self.poststed = poststed
        
    def __lt__ (self, other):
        this = self.etternavn + self.fornavn
        other = other.etternavn + other.fornavn
        return this < other

    def __gt__ (self, other):
        this = self.etternavn + self.fornavn
        other = other.etternavn + other.fornavn
        return this > other

    def __eq__ (self, other):
        this = self.etternavn + self.fornavn
        other = other.etternavn + other.fornavn
        return this == other
        
    def __str__(self):
        return f"{self.etternavn};{self.fornavn};{self.adresse};{self.postnummer};{self.poststed}"
    

In [2]:
persons = []
with open("Personer.dta") as file:
    for line in file:
        a = line.strip().split(";")
        persons.append(Person(*a))

for x in persons[-5:]:
    print(x)

STORLIEN;HANS-MAGNUS;TALLSLETTA 57;3849;VRÅLIOSEN
ASAIPPILLAI;JO GUDBRAND;BERGERUD 68;4371;EGERSUND
TOLLEFSEN;ZULFIQAR ALI;EIRIK JARLS GATE 79;1305;HASLUM
ELI;KNUT OLAF;RUSTAD S 14;9775;GAMVIK
NYRUD;ERIK NORØ;GJERDHAUGVEGEN 3;6512;KRISTIANSUND N


In [3]:
len(set([x.postnummer for x in persons]))

2824

In [4]:
%%timeit -r 1 -n 1
#Takes a list and counts number of appearances in form of a dictonary. Key=element Value=appearances
def appearances_in_list(lst: list) -> dict:
    appearances = dict()
    for x in lst:
        try:
            appearances[x] += 1
        except:
            appearances[x] = 0
    return appearances
    
#Baiscally selection sort if n=len(dict)
def n_biggest_keys(dct: dict, n: int = 1) -> dict:
    if n < 1:
        return {}
    else:
        biggest = max_dict(dct)
        k = dct.pop(biggest)
        n -= 1
        r = {biggest: k}
        r.update(n_biggest_keys(dct, n))
        return r

#Finds the largest value from the dictonary and returns the key of said value. 
def max_dict(dct: dict) -> str:
    biggest = next(iter(dct))
    for x in dct:
        if dct.get(x) > dct.get(biggest):
            biggest = x
    return biggest

#finds n most common class attributes in a list and returns a list containing sorted dictonaries.
#Entering -1 for n returns the whole dictornary sorted.
def mcail(lst: list, attribute: str, n: int) -> list:
    list_of_attributes = [getattr(x, attribute) for x in lst]
    apps = appearances_in_list(list_of_attributes)
    if n < 0:
        n = len(apps)
    return n_biggest_keys(apps, n)

tmcl = mcail(persons, "etternavn", 10) 
for i, k in enumerate(tmcl.items()):
    print(f"{i+1}. {k[0]} {k[1]}")

1. ÅSBAKK 82
2. BERGERSEN 77
3. DANIELSEN 76
4. LUTHER 76
5. TRANE 76
6. LUNDBERGSEN 76
7. MJØEN 75
8. LAM 74
9. KRANGSÅS 74
10. IGLAND 74
48.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%%timeit -r 1 -n 1
#alternative 1
etternavn2 = [x.etternavn for x in persons]
#Method taken from https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/
def most_frequent(List):
    dict = {}
    count, itm = 0, ''
    for item in reversed(List):
        dict[item] = dict.get(item, 0) + 1
        if dict[item] >= count :
            count, itm = dict[item], item
    return(itm)

def n_most_common2(lst, n):
    top_n = []
    counter=0
    while counter < n:
        mf = most_frequent(lst)
        top_n.append(mf)
        list(filter((top_n[counter]).__ne__, lst))
        lst = [x for x in lst if x != top_n[counter]]
        counter+=1
        
    return top_n
print(n_most_common2(etternavn2, 10))

['ÅSBAKK', 'BERGERSEN', 'DANIELSEN', 'LUTHER', 'TRANE', 'LUNDBERGSEN', 'MJØEN', 'LAM', 'KRANGSÅS', 'IGLAND']
528 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
%%script false 
#alternative 2
#extremely slow for some reason and therefore disabled.
etternavn = [x.etternavn for x in persons]
def n_most_common(lst, n):
    top_n = []
    counter=0
    while counter < n:
        top_n.append(max(set(lst), key=lst.count))
        list(filter((top_n[counter]).__ne__, lst))
        lst = [x for x in lst if x != top_n[counter]]
        counter+=1
        
    return top_n
#The last names with the same values may not be in the same order as the one above.
print(n_most_common(etternavn, 10))

Couldn't find program: 'false'


In [7]:
%%timeit -r 1 -n 1
def heap_sort(lst: list) -> list:
    heap_size = 1
    l = len(lst)
    while heap_size < l:
        add_to_heap(lst, heap_size)
        heap_size += 1
    while heap_size > 0:
        remove_from_heap(lst, heap_size-1)
        heap_size -= 1
    return lst    

#In a list..
def swap_elements(lst: list, i: int, j: int) -> list:
    lst[i], lst[j] = lst[j], lst[i]
    return lst

#presumes that the list before index i is a complete max heap.
#Adds the index to the heap and makes sure it retains the properties of a heap.
def add_to_heap(heap: list, i: int) -> list:
    parent = int((i-1)/2)
    if i < 1 or heap[i] < heap[parent]:
        return heap
    else:
        swap_elements(heap, i, parent)
        return add_to_heap(heap, parent)
    
#Always removes the top element and puts it after the heap.
#Then calls heapify_index to recreate a maxheap
#The last variable refers to the index of the last element in the heap.
def remove_from_heap(heap: list, last: int) -> list:
    swap_elements(heap, 0, last)
    heapify_index(heap, last-1, 0)
    return heap
    
#Takes the index of an element in a binary tree and checks if its children are bigger.
#If one of them are it swaps places with the bigger element.
#Then recursively checks its chieldren again.
def heapify_index(heap: list, last: int, i: int) -> list:
    firstborn = 2*i+1 #1
    secondborn = firstborn+1 #2
    if firstborn > last: 
        return heap
    elif secondborn > last:
        if heap[firstborn] > heap[i]:
            swap_elements(heap, i, firstborn)
        return heap
    else:
        bigger_element = secondborn if heap[firstborn] < heap[secondborn] else firstborn
        if heap[bigger_element] > heap[i]:
            swap_elements(heap, i, bigger_element)
            return heapify_index(heap, last, bigger_element)
        else:
            return heap
#could be used to create a heap? 

intlist = []
import random
for x in range(0, 30):
    intlist.append(random.randint(0,20))
heap_sort(intlist)
print(intlist)
    
heap_sort(persons)
for x in persons[::20000]:
    print(x)

[0, 1, 1, 1, 2, 2, 4, 4, 5, 6, 6, 6, 7, 8, 8, 9, 9, 9, 9, 11, 12, 12, 13, 15, 15, 17, 18, 18, 18, 20]
AAKVIK;ANETTE;BAKLIEN 11;1360;NESBRU
ENGVOLL;TORE REIDAR;DAGALIVEIEN 11;3623;LAMPELAND
JERMOSEN;ÅGE RAYMOND;HATLEBAKKANE 6;5236;RÅDAL
MYRLUND;GAUTE;DYNESTØL 89;7494;SINGSÅS
SMITH;ARNT OLE;SAPPEN 39;5852;BERGEN
3.11 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [8]:
%%timeit -r 1 -n 1
#Second attempt
def se(lst: list, i: int, j: int) -> list:
    lst[i], lst[j] = lst[j], lst[i]
    return lst

#Does the same as the "heapify_index" from the cell above.
def heapify(heap: list, last: int, i: int) -> list:
    f = 2*i+1 #1
    s = f+1 #2
    if f > last:
        return heap
    if s > last:
        if heap[f] > heap[i]:
            se(heap, f, i)
        return heap
    m = s if heap[f] < heap[s] else f
    if heap[m] > heap[i]:
        se(heap, m, i)
        return heapify(heap, last, m)
    return heap

#This time around the plan is to use the heapify method to create the max heap.
#To achieve this we need to use it from the bottom up
def heap_sort(lst: list) -> list:
    l = len(lst)-1
    #We can start from the 2nd to last layer. 
    #This finds the parent element of the last element
    s = int((l-1)/2)
    #Heapifies every element before the parent of the last element.
    while s >= 0:
        heapify(lst, l, s)
        s -= 1
    #Swapps the first and last then heapifys the first elements till the new last element.
    #Similar to the cell above
    while l > 0:
        se(lst, 0, l)
        heapify(lst, l-1, 0)
        l -= 1
    return lst
        
intlist = []
import random
for x in range(0, 30):
    intlist.append(random.randint(0,20))
heap_sort(intlist)
print(intlist)
    
heap_sort(persons)
for x in persons[::20000]:
    print(x)

[1, 2, 3, 3, 4, 4, 4, 5, 7, 7, 8, 8, 9, 10, 10, 11, 11, 11, 12, 13, 13, 14, 14, 14, 15, 18, 18, 18, 19, 20]
AAKVIK;ANETTE;BAKLIEN 11;1360;NESBRU
ENGVOLL;TORE REIDAR;DAGALIVEIEN 11;3623;LAMPELAND
JERMOSEN;ÅGE RAYMOND;HATLEBAKKANE 6;5236;RÅDAL
MYRLUND;GAUTE;DYNESTØL 89;7494;SINGSÅS
SMITH;ARNT OLE;SAPPEN 39;5852;BERGEN
2.81 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
